In [3]:
def compileQuery(query, vars):
    repl = {}
    for var in vars:
        repl["{$" + str(var) + "}"] = vars[var]
    qc = query
    for match in repl:
        qc = qc.replace(match, repl[match])
    return str(qc)

def getCSVOutputCommand(query, filename):
    return "USE sqlrest_baselearners;\n\n" + query + "\nINTO OUTFILE '" + filename + "'       FIELDS TERMINATED BY ';' ENCLOSED BY '\"' LINES TERMINATED BY '\\n';"

# Queries for Atomic Algorithm Performances

In [18]:
queries = {
    "progressquery": "SELECT '{$ALGORITHM}' as algorithm, open, CONCAT(ROUND(100 * open / total, 2), '%') as 'open (rel)', running, CONCAT(ROUND(100 * running / total, 2), '%') as 'running (rel)', finished, CONCAT(ROUND(100 * finished / total, 2), '%') as 'finished (rel)', failed, total, CONCAT(ROUND(avgRuntimeFinished), 's') as 'Average Time of Finished', CONCAT(ROUND(avgRuntimeFinished * open / running), 's') as 'ETA' FROM (SELECT 'aux' as pk, COUNT(*) as 'open' FROM `evaluations_classifiers_{$ALGORITHM}` WHERE time_started is null) as t1 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as 'running' FROM `evaluations_classifiers_{$ALGORITHM}` WHERE time_started is not null and time_end is null) as t2 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as finished, AVG(TIMESTAMPDIFF(SECOND, time_started, time_end)) as avgRuntimeFinished  FROM `evaluations_classifiers_{$ALGORITHM}` WHERE time_started is not null and time_end is not null) as t3 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as failed FROM `evaluations_classifiers_{$ALGORITHM}` where exception is not null) as t4 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as total FROM `evaluations_classifiers_{$ALGORITHM}`) as t5",
    "basicresults": "SELECT openmlid, datapoints + predictedinstances as totalsize, datapoints as fitsize, predictedinstances as applicationsize, seed, '{$ALGORITHM}' as classifier, '' as algorithmoptions, numattributes, numberofcategories, numericattributesafterbinarization, numlabels, numnumericattributes, numsymbolicattributes, attributestocover50pctvariance, attributestocover90pctvariance, attributestocover95pctvariance, attributestocover99pctvariance, totalvariance, traintimeinms as fittime, timeforpredictionsinms as applicationtime, stdinpredictiontimeminms, replace(replace(exception, \"\\n\", \"\\\\n\"), \"\\\"\", \"\") as exception FROM `evaluations_classifiers_{$ALGORITHM}` where time_end is not null"
}

In [22]:
jointQuery = "\nUNION\n".join([compileQuery(queries["basicresults"], {"ALGORITHM": a}) for a in ["bayesnet", "ibk", "j48", "kstar"]])
print(getCSVOutputCommand(jointQuery, "classifierresults-default.csv"))

USE sqlrest_baselearners;

SELECT openmlid, datapoints + predictedinstances as totalsize, datapoints as fitsize, predictedinstances as applicationsize, seed, 'bayesnet' as classifier, '' as algorithmoptions, numattributes, numberofcategories, numericattributesafterbinarization, numlabels, numnumericattributes, numsymbolicattributes, attributestocover50pctvariance, attributestocover90pctvariance, attributestocover95pctvariance, attributestocover99pctvariance, totalvariance, traintimeinms as fittime, timeforpredictionsinms as applicationtime, stdinpredictiontimeminms, replace(replace(exception, "\n", "\\n"), "\"", "") as exception FROM `evaluations_classifiers_bayesnet` where time_end is not null
UNION
SELECT openmlid, datapoints + predictedinstances as totalsize, datapoints as fitsize, predictedinstances as applicationsize, seed, 'ibk' as classifier, '' as algorithmoptions, numattributes, numberofcategories, numericattributesafterbinarization, numlabels, numnumericattributes, numsymboli

In [21]:
jointQuery = "\nUNION\n".join([compileQuery(queries["progressquery"], {"ALGORITHM": a}) for a in ["bayesnet", "ibk", "j48", "kstar"]])
#print(getCSVOutputCommand(jointQuery, "classifierresults-default.csv"))
print(jointQuery)

SELECT 'bayesnet' as algorithm, open, CONCAT(ROUND(100 * open / total, 2), '%') as 'open (rel)', running, CONCAT(ROUND(100 * running / total, 2), '%') as 'running (rel)', finished, CONCAT(ROUND(100 * finished / total, 2), '%') as 'finished (rel)', failed, total, CONCAT(ROUND(avgRuntimeFinished), 's') as 'Average Time of Finished', CONCAT(ROUND(avgRuntimeFinished * open / running), 's') as 'ETA' FROM (SELECT 'aux' as pk, COUNT(*) as 'open' FROM `evaluations_classifiers_bayesnet` WHERE time_started is null) as t1 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as 'running' FROM `evaluations_classifiers_bayesnet` WHERE time_started is not null and time_end is null) as t2 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as finished, AVG(TIMESTAMPDIFF(SECOND, time_started, time_end)) as avgRuntimeFinished  FROM `evaluations_classifiers_bayesnet` WHERE time_started is not null and time_end is not null) as t3 NATURAL JOIN (SELECT 'aux' as pk, COUNT(*) as failed FROM `evaluations_classifiers_bayesnet` wh

# Queries for Results of ML-Plan

In [2]:
def compileQuery(query, to, mode):
    repl = {
        '{$T_EVAL}': VARS[to][mode]["eval"],
        '{$T_JOBS}': VARS[to][mode]["jobs"],
        '{$TARGET}': mode
    }
    qc = queries[query]
    for match in repl:
        qc = qc.replace(match, repl[match])
    return str(qc)

def compileJoin(query, to):
    q1 = compileQuery(query, to, "vanilla")
    q2 = compileQuery(query, to, "guarded")
    return "SELECT * FROM (" + q1 + ") as t1 JOIN (" + q2 + ") as t2 USING(dataset)"

In [8]:
VARS = {
    '1h': {
        'vanilla': {
            'eval': 'cont_eval_mlplan_1h',
            'jobs': 'cont_jobs_mlplan_1h'
        },
        'guarded': {
            'eval': 'cont_eval_mlplan_safeguard_1h',
            'jobs': 'cont_jobs_mlplan_safeguard_1h',
        } 
    },
    '1d': {
        'vanilla': {
            'eval': 'cont_eval_mlplan_24h',
            'jobs': 'cont_jobs_mlplan_24h'
        },
        'guarded': {
            'eval': 'cont_eval_mlplan_safeguard_24h',
            'jobs': 'cont_jobs_mlplan_safeguard_24h',
        } 
    }
}

In [27]:
queries = {
    "avgSuccessCounts": 'SELECT dataset, avg(n) as n_{$TARGET}, std(n) as std_{$TARGET} FROM (SELECT experiment_id, COUNT(*) as n FROM `{$T_EVAL}` WHERE status = "success" AND thread LIKE "%worker%" GROUP by experiment_id) as t natural join {$T_JOBS}  WHERE time_end IS NOT NULL group by dataset',
    "avgSuccessCountsPerThread": 'SELECT dataset, avg(n) as n_{$TARGET} FROM (SELECT experiment_id, COUNT(*) as n FROM `{$T_EVAL}` WHERE status = "success" AND thread LIKE "%worker%" GROUP by experiment_id, thread) as t natural join {$T_JOBS}  WHERE time_end IS NOT NULL group by dataset',
    "avgSuccessTimes": 'SELECT dataset, avg(successtime) as t_{$TARGET} FROM (SELECT experiment_id, dataset, sum(actualFitTime+ actualPredictTime) as successtime FROM `{$T_EVAL}` natural join {$T_JOBS} WHERE time_end IS NOT NULL AND result IS NOT NULL AND status = "success" GROUP BY experiment_id) as t GROUP BY dataset',
    "avgSuccessTimesPerThread": 'SELECT dataset, AVG(successtime) as t_{$TARGET} FROM( SELECT experiment_id, thread, dataset, sum(actualFitTime+ actualPredictTime) * 5 as successtime FROM {$T_EVAL} natural join {$T_JOBS} WHERE time_end IS NOT NULL AND result IS NOT NULL AND status = "success" and thread LIKE "%worker%" GROUP BY experiment_id, thread) as t2 GROUP BY dataset',
    "avgTimeouts": 'SELECT * FROM (SELECT dataset, avg(timeoutedevaluations) as timeouts_{$TARGET} FROM (SELECT experiment_id, COUNT(*) as timeoutedevaluations FROM `{$T_EVAL}` WHERE status = "timeout" and actualFitTime + actualPredictTime > 60 AND thread LIKE "%worker%" GROUP BY experiment_id) as t NATURAL JOIN {$T_JOBS} WHERE time_end IS NOT NULL group by dataset) as t2',
    "avgAvoidances": 'SELECT dataset, avg(n) as n_{$TARGET} FROM (SELECT experiment_id, COUNT(*) as n FROM `{$T_EVAL}` WHERE status = "safeguard" GROUP by experiment_id) as t natural join {$T_JOBS}  WHERE time_end IS NOT NULL group by dataset',
    "avgWorkerTime": 'SELECT  dataset, avg(avgworktime) FROM (SELECT experiment_id, dataset, avg(worktime) as avgworktime FROM (SELECT experiment_id, dataset, thread, round((MAX(timestamp_found) - min(timestamp_found)) / 1000) as worktime FROM `{$T_EVAL}` natural join {$T_JOBS} where thread LIKE "%worker%" GROUP BY experiment_id, thread) as t group by experiment_id) as t2 group by dataset'
}

# Run Analysis Queries

In [28]:
print(compileJoin("avgSuccessCounts", "1h"))

SELECT * FROM (SELECT dataset, avg(n) as n_vanilla, std(n) as std_vanilla FROM (SELECT experiment_id, COUNT(*) as n FROM `cont_eval_mlplan_1h` WHERE status = "success" AND thread LIKE "%worker%" GROUP by experiment_id) as t natural join cont_jobs_mlplan_1h  WHERE time_end IS NOT NULL group by dataset) as t1 JOIN (SELECT dataset, avg(n) as n_guarded, std(n) as std_guarded FROM (SELECT experiment_id, COUNT(*) as n FROM `cont_eval_mlplan_safeguard_1h` WHERE status = "success" AND thread LIKE "%worker%" GROUP by experiment_id) as t natural join cont_jobs_mlplan_safeguard_1h  WHERE time_end IS NOT NULL group by dataset) as t2 USING(dataset)


In [11]:
print(compileJoin("avgSuccessTimes", "1d"))

SELECT * FROM (SELECT dataset, avg(successtime) as t_vanilla FROM (SELECT experiment_id, dataset, sum(actualFitTime+ actualPredictTime) as successtime FROM `cont_eval_mlplan_24h` natural join cont_jobs_mlplan_24h WHERE time_end IS NOT NULL AND result IS NOT NULL AND status = "success" GROUP BY experiment_id) as t GROUP BY dataset) as t1 JOIN (SELECT dataset, avg(successtime) as t_guarded FROM (SELECT experiment_id, dataset, sum(actualFitTime+ actualPredictTime) as successtime FROM `cont_eval_mlplan_safeguard_24h` natural join cont_jobs_mlplan_safeguard_24h WHERE time_end IS NOT NULL AND result IS NOT NULL AND status = "success" GROUP BY experiment_id) as t GROUP BY dataset) as t2 USING(dataset)


In [6]:
print(compileJoin("avgWorkerTime", "1h"))

SELECT * FROM (SELECT  dataset, avg(avgworktime) FROM (SELECT experiment_id, dataset, avg(worktime) as avgworktime FROM (SELECT experiment_id, dataset, thread, round((MAX(timestamp_found) - min(timestamp_found)) / 1000) as worktime FROM `cont_eval_mlplan_1h` natural join cont_jobs_mlplan_1h where thread LIKE "%worker%" GROUP BY experiment_id, thread) as t group by experiment_id) as t2 group by dataset) as t1 JOIN (SELECT  dataset, avg(avgworktime) FROM (SELECT experiment_id, dataset, avg(worktime) as avgworktime FROM (SELECT experiment_id, dataset, thread, round((MAX(timestamp_found) - min(timestamp_found)) / 1000) as worktime FROM `cont_eval_mlplan_safeguard_1h` natural join cont_jobs_mlplan_safeguard_1h where thread LIKE "%worker%" GROUP BY experiment_id, thread) as t group by experiment_id) as t2 group by dataset) as t2 USING(dataset)


# Relevant Queries for Final Results

## Number of Avoided Executions

In [55]:
def getQueryForAvgAvoidances(to):
       return compileQuery("avgAvoidances", to, "guarded")

In [56]:
print(getQueryForAvgAvoidances("1h"))

SELECT dataset, avg(n) as n_guarded FROM (SELECT experiment_id, COUNT(*) as n FROM `cont_eval_mlplan_safeguard_1h` WHERE status = "safeguard" GROUP by experiment_id) as t natural join cont_jobs_mlplan_safeguard_1h  WHERE time_end IS NOT NULL group by dataset


## Number of Timeouts Per Run

In [47]:
def getQueryForAvgTimeouts(to):
       return compileJoin("avgTimeouts", to)

In [48]:
print(getQueryForAvgTimeouts("1h"))

SELECT * FROM (SELECT * FROM (SELECT dataset, avg(timeoutedevaluations) as timeouts_vanilla FROM (SELECT experiment_id, COUNT(*) as timeoutedevaluations FROM `cont_eval_mlplan_1h` WHERE status = "timeout" and actualFitTime + actualPredictTime > 60 AND thread LIKE "%worker%" GROUP BY experiment_id) as t NATURAL JOIN cont_jobs_mlplan_1h WHERE time_end IS NOT NULL group by dataset) as t2) as t1 JOIN (SELECT * FROM (SELECT dataset, avg(timeoutedevaluations) as timeouts_guarded FROM (SELECT experiment_id, COUNT(*) as timeoutedevaluations FROM `cont_eval_mlplan_safeguard_1h` WHERE status = "timeout" and actualFitTime + actualPredictTime > 60 AND thread LIKE "%worker%" GROUP BY experiment_id) as t NATURAL JOIN cont_jobs_mlplan_safeguard_1h WHERE time_end IS NOT NULL group by dataset) as t2) as t2 USING(dataset)


## Number of Successful Executions Per Run

In [45]:
def getQueryForAvgSuccess(to):
       return compileJoin("avgSuccess", to)

In [46]:
print(getQueryForAvgSuccess("1h"))

SELECT * FROM (SELECT dataset, avg(n) as n_vanilla FROM (SELECT experiment_id, COUNT(*) as n FROM `cont_eval_mlplan_1h` WHERE status = "success" GROUP by experiment_id) as t natural join cont_jobs_mlplan_1h  WHERE time_end IS NOT NULL group by dataset) as t1 JOIN (SELECT dataset, avg(n) as n_guarded FROM (SELECT experiment_id, COUNT(*) as n FROM `cont_eval_mlplan_safeguard_1h` WHERE status = "success" GROUP by experiment_id) as t natural join cont_jobs_mlplan_safeguard_1h  WHERE time_end IS NOT NULL group by dataset) as t2 USING(dataset)


In [61]:
print(compileQuery("avgSuccess", "1h", "guarded"))

SELECT dataset, avg(n) as n_guarded FROM (SELECT experiment_id, COUNT(*) as n FROM `cont_eval_mlplan_safeguard_1h` WHERE status = "success" GROUP by experiment_id) as t natural join cont_jobs_mlplan_safeguard_1h  WHERE time_end IS NOT NULL group by dataset


## Number of Executions Grouped by Type

In [ ]:
SELECT dataset, status, avg(c)  FROM (SELECT experiment_id, status, count(*) as c FROM `cont_eval_mlplan_1h` GROUP BY experiment_id, status) as t natural join cont_jobs_mlplan_1h group by dataset, status

## Avg Time Spent in Executions of different Status

## Avg time used by the workers

In [ ]:
SELECT dataset, avg(worktime) FROM (SELECT experiment_id, dataset, thread, round((MAX(timestamp_found) - min(timestamp_found)) / 1000) as worktime FROM `cont_eval_mlplan_safeguard_1h` natural join cont_jobs_mlplan_safeguard_1h where thread LIKE "%worker%" GROUP BY experiment_id, thread) as t GROUP BY dataset

## Get experiments with avg worker time < x

In [ ]:
SELECT experiment_id, dataset, avgworktime FROM (SELECT experiment_id, dataset, avg(worktime) as avgworktime FROM (SELECT experiment_id, dataset, thread, round((MAX(timestamp_found) - min(timestamp_found)) / 1000) as worktime FROM `cont_eval_mlplan_safeguard_1h` natural join cont_jobs_mlplan_safeguard_1h where thread LIKE "%worker%" GROUP BY experiment_id, thread) as t group by experiment_id) as t2 WHERE avgworktime < 2500 